In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

import pickle

from scipy import spatial

In [2]:
df = pd.read_csv('Complaints_noise.csv')

In [3]:
df.shape

(1086650, 16)

In [4]:
df.tail()

,Unnamed: 0,address_type,borough,city,complaint_type,cross_street_1,cross_street_2,facility_type,incident_address,incident_zip,intersection_street_1,intersection_street_2,latitude,longitude,unique_key,call_date
1086645,1086645,ADDRESS,QUEENS,RIDGEWOOD,Noise - Residential,DECATUR STREET,CODY AVENUE,Precinct,1111 WYCKOFF AVENUE,11385.0,NaN,NaN,40.694001,-73.901408,34554183,2016-10-16 04:42:35
1086646,1086646,ADDRESS,BROOKLYN,BROOKLYN,Noise - Residential,GLENWOOD ROAD,FLATLANDS AVENUE,Precinct,715 EAST 102 STREET,11236.0,NaN,NaN,40.647282,-73.898216,34554184,2016-10-16 03:41:51
1086647,1086647,ADDRESS,BRONX,BRONX,Noise - Residential,EAST 184 STREET,EAST 187 STREET,Precinct,2386 HOFFMAN STREET,10458.0,NaN,NaN,40.855580,-73.888824,34554198,2016-10-16 06:31:03
1086648,1086648,ADDRESS,QUEENS,ELMHURST,Noise - Residential,ELMHURST AVENUE,WHITNEY AVENUE,Precinct,40-82 DENMAN STREET,11373.0,NaN,NaN,40.746969,-73.875594,34554199,2016-10-16 02:52:36
1086649,1086649,ADDRESS,BROOKLYN,BROOKLYN,Noise - Residential,GLENWOOD ROAD,FLATLANDS AVENUE,Precinct,707 EAST 102 STREET,11236.0,NaN,NaN,40.647326,-73.898263,34554202,2016-10-16 03:56:58


In [5]:
dff = df.drop(['Unnamed: 0', 
               'address_type', 
               'borough',
               'city',
               'cross_street_1',
               'cross_street_2', 
               'incident_address',
               'incident_zip',
               'intersection_street_1', 
               'intersection_street_2'],axis=1)

In [6]:
dff.head()

,complaint_type,facility_type,latitude,longitude,unique_key,call_date
0,Noise - Residential,Precinct,40.828783,-73.921617,34037643,2016-08-09 00:43:14
1,Noise - Residential,Precinct,40.831527,-73.928863,34042959,2016-08-09 02:01:49
2,Noise - Residential,Precinct,40.837573,-73.918969,34043216,2016-08-09 23:39:28
3,Noise - Residential,Precinct,40.830346,-73.923274,34044247,2016-08-10 00:22:07
4,Noise - Residential,Precinct,40.833263,-73.918862,34045382,2016-08-10 00:37:43


In [7]:
dff['coordinates'] = list(zip(dff.latitude, dff.longitude))
dff.coordinates = dff.coordinates.apply(list)

In [8]:
with open('df_noise.pkl','wb') as picklefile:
    pickle.dump(dff, picklefile)

with open('df_noise.pkl', 'rb') as picklefile:
    dff = pickle.load(picklefile)

dff.shape

(1086650, 7)

In [9]:
with open('df_noise.pkl', 'rb') as picklefile:
    dff = pickle.load(picklefile)

dff.shape

(1086650, 7)

In [10]:
df_ct = pd.read_csv('geoid_census_tract.csv')
df_ct.shape

(2168, 7)

In [11]:
df_ct['COORDS'] = list(zip(df_ct.LAT, df_ct.LON))
df_ct.COORDS = df_ct.COORDS.apply(list)
df_ct.head()

,INDEX,GEOID,STATE,COUNTY,TRACT,LAT,LON,COORDS
0,88,36005000100,36,5,100,40.793492,-73.883532,"[40.7934921, -73.8835318]"
1,89,36005000200,36,5,200,40.804573,-73.856859,"[40.8045733, -73.8568585]"
2,90,36005000400,36,5,400,40.808915,-73.850488,"[40.8089152, -73.8504884]"
3,91,36005001600,36,5,1600,40.818848,-73.858076,"[40.8188478, -73.8580764]"
4,92,36005001900,36,5,1900,40.800999,-73.909373,"[40.800999, -73.9093729]"


In [12]:
X_train = pd.Series(df_ct.COORDS).tolist()
y_train = df_ct.GEOID
X_test = [dff.coordinates]
len(X_test[0])

1086650

In [13]:
def nearest(X_train, y_train, X_test):
    tree = spatial.KDTree(np.array(X_train))
    (distances,indexes) = tree.query(X_test)
    for i,d in list(zip(indexes,distances)):
        yield y_train[i]

In [14]:
results = []
for result in nearest(X_train, y_train, X_test):
    results.append(result)
len(results[0])

1086650

In [15]:
df_coords = pd.DataFrame(np.asarray(results))
df_coords = df_coords.transpose()

In [16]:
with open('df_coords_noise.pkl','wb') as picklefile:
    pickle.dump(df_coords, picklefile)

with open('df_coords_noise.pkl', 'rb') as picklefile:
    df_coords = pickle.load(picklefile)

df_coords.head()

,0
0,3.600502e+10
1,3.600502e+10
2,3.600502e+10
3,3.600502e+10
4,3.600502e+10


In [17]:
with open('df_coords_noise.pkl', 'rb') as picklefile:
    df_coords = pickle.load(picklefile)

In [18]:
df_coords = df_coords.rename(columns={0:'census_tract'})
df_coords.head()
df_coords['census_tract'] = df_coords['census_tract'].apply(str)
df_coords.head()

,census_tract
0,36005018301.0
1,36005018900.0
2,36005022101.0
3,36005019500.0
4,36005018102.0


In [19]:
dff['census_tract'] = df_coords

In [20]:
dff['complaint_count'] = 1

In [21]:
dff.shape

(1086650, 9)

In [22]:
df_g = dff.groupby('census_tract',as_index=False).sum()

In [23]:
df_g_ = df_g.drop(['latitude','longitude','unique_key'],axis=1)

In [24]:
df_g_.head()

,census_tract,complaint_count
0,36005000200.0,170
1,36005000400.0,244
2,36005001600.0,548
3,36005001900.0,13
4,36005002000.0,1290


In [25]:
df_g_.replace('NaN',np.nan)
df_ = df_g_.fillna(value=0)

In [33]:
df_['census_tract'] = df_['census_tract'].astype(float)

In [36]:
df_.head()

,census_tract,complaint_count
0,3.600500e+10,170
1,3.600500e+10,244
2,3.600500e+10,548
3,3.600500e+10,13
4,3.600500e+10,1290


In [41]:
with open('noisecomplaint_for_model.pkl','wb') as picklefile:
    pickle.dump(df_, picklefile)

In [42]:
with open('noisecomplaint_for_model.pkl', 'rb') as picklefile:
    noise = pickle.load(picklefile)

In [43]:
noise.dtypes

census_tract       float64
complaint_count      int64
dtype: object